In [6]:
import requests

response = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/les-1000-titres-les-plus-reserves-dans-les-bibliotheques-de-pret/records?limit=100")

# Afficher le type de contenu et le texte brut de la réponse
print("Type de contenu :", response.headers.get("Content-Type"))

Type de contenu : application/json; charset=utf-8


On met comme limite 100 pcq si on ne le fait pas, il y a un message d'erreur qui dit que la limite doit être entre -1 et 100

In [7]:
wb = response.json()  # Utilisation de .json() sur l'objet réponse
print("Structure du JSON :")
print(wb)

Structure du JSON :
{'total_count': 1000, 'results': [{'rang': 12, 'type_de_document': 'Livre adulte', 'reservations': 522.0, 'titre': "Réinventer l'amour : comment le patriarcat sabote les relations hétérosexuelles", 'auteur': 'Chollet,  Mona'}, {'rang': 13, 'type_de_document': 'Bande dessinée adulte', 'reservations': 521.0, 'titre': 'Le jeune acteur', 'auteur': 'Sattouf,  Riad'}, {'rang': 14, 'type_de_document': 'Bande dessinée adulte', 'reservations': 510.0, 'titre': 'Une jeunesse au Moyen-Orient, 1992-1994', 'auteur': 'Sattouf,  Riad'}, {'rang': 37, 'type_de_document': 'DVD tous publics', 'reservations': 337.0, 'titre': 'Dune', 'auteur': None}, {'rang': 58, 'type_de_document': 'DVD tous publics', 'reservations': 266.0, 'titre': 'Onoda. 10 000 nuits dans la jungle', 'auteur': None}, {'rang': 73, 'type_de_document': 'Nouveauté', 'reservations': 241.0, 'titre': 'Chien 51 : roman', 'auteur': 'Gaudé,  Laurent'}, {'rang': 87, 'type_de_document': 'Livre adulte', 'reservations': 225.0, 'ti

Question: pourquoi est-ce qu'il y a pas tous les rangs ???

In [8]:
import pandas as pd
df= pd.json_normalize(wb['results'])
df.head(12)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
3,37,DVD tous publics,337.0,Dune,None
4,58,DVD tous publics,266.0,Onoda. 10 000 nuits dans la jungle,None
5,73,Nouveauté,241.0,Chien 51 : roman,"Gaudé, Laurent"
6,87,Livre adulte,225.0,Le gosse : roman,"Olmi, Véronique"
7,109,Bande dessinée jeunesse,196.0,Attaque au clair de lune,"Oda, Eiichiro"
8,125,DVD nouveautés tous publics,177.0,West Side Story,None
9,136,None,170.0,Etés anglais,"Howard, Elizabeth Jane"


In [9]:
print(df["type_de_document"].unique())

['Livre adulte' 'Bande dessinée adulte' 'DVD tous publics' 'Nouveauté'
 'Bande dessinée jeunesse' 'DVD nouveautés tous publics' None
 'Nouveauté jeunesse' 'Bande dessinée ado' 'Livre ado' 'Livre jeunesse']


In [10]:
doc = ["Livre adulte", "Bande dessinée adulte", "Nouveauté", "Bande dessinée jeunesse", "Nouveauté jeunesse",
 "Bande dessinée ado", "Livre ado", "Livre jeunesse"]
df_filtered= df[df["type_de_document"].isin(doc)]

In [11]:
df_filtered.head(5)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
5,73,Nouveauté,241.0,Chien 51 : roman,"Gaudé, Laurent"
6,87,Livre adulte,225.0,Le gosse : roman,"Olmi, Véronique"


In [12]:
nouveau = ["Nouveauté", "Nouveauté jeunesse"]
df_nouveau =  df[df["type_de_document"].isin(nouveau)]
print(df_nouveau)

    rang    type_de_document  reservations  \
5     73           Nouveauté         241.0   
19   394  Nouveauté jeunesse          95.0   
20   399           Nouveauté          95.0   
22   425  Nouveauté jeunesse          91.0   
34   672           Nouveauté          71.0   
40   720  Nouveauté jeunesse          68.0   
49   910           Nouveauté          60.0   
51   945           Nouveauté          58.0   
52   949           Nouveauté          58.0   
66   217           Nouveauté         132.0   
69   233           Nouveauté         127.0   
71   275           Nouveauté         114.0   
92   726           Nouveauté          68.0   
96   762           Nouveauté          66.0   

                                                titre                 auteur  
5                                    Chien 51 : roman        Gaudé,  Laurent  
19      My hero academia. 8. Momo Yaoyorozu : l'envol      Horikoshi,  Kohei  
20                                 Le poids des héros           Sala,  D

Le but maintenant va être de scrapper wikipedia pour compléter la base et les catégories "nouveaux" qui ne renseignent pas sur le type d'ouvrage

In [13]:
import requests
from bs4 import BeautifulSoup

# Fonction pour rechercher un livre sur Babelio et accéder à la page du premier résultat
def search_livraddict(book_title):
    # URL de recherche sur Livraddict
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_url = f'https://www.livraddict.com/search.php?t={book_title}]'

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    # Trouver le premier élément correspondant au lien d'un résultat
    page = BeautifulSoup(response, "html.parser")
    first_result = page.select_one('.listing_recherche li .item_photo a')
    if first_result is not None:
        url = first_result['href']
        full_url = f"https://www.livraddict.com{url}"
        return full_url
    else:
        return None

In [14]:
def obtain_type(url):
    if url is not None:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers).content
        page = BeautifulSoup(response, "html.parser")
        c_format= page.find("div", class_= "book_classif book_format mr1")
        c_lectorat= page.find("div", class_="book_classif book_lectorat mr1")
        if c_format:
            a_format = c_format.find("a")
            format_livre = a_format.text
        else:
            format_livre = "Non renseigné"
        if c_lectorat:
            a_lectorat = c_lectorat.find("a")
            lectorat = a_lectorat.text
        else: 
            lectorat = "Non renseigné"
        info = [format_livre, lectorat]
        return info
    else:
        return None

In [15]:
def type_livre(info):
    if info is None:
        type_livre="Nom du livre pas trouvé sur Livraddict"
    else:
        # Normaliser les chaînes (supprimer les espaces et convertir en minuscules)
        info_0_normalized = info[0].strip().lower()
        info_1_normalized = info[1].strip().lower()

        if info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["petite enfance", "enfance"]:
            type_livre = "Bande dessinée jeunesse"

        elif info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["adolescence"]:
            type_livre = "Bande dessinée ado"

        elif info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["young adult", "adulte"]:
            type_livre = "Bande dessinée ado"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["petite enfance", "enfance"]:
            type_livre = "Livre jeunesse"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["adolescence"]:
            type_livre = "Livre ado"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["young adult", "adulte"]:
            type_livre = "Livre adulte"

        else: type_livre= "Introuvable"
    return type_livre


In [16]:
def search_type_livre(titre):
    livre = titre.split()
    titre_livre = ""
    for i in range(0, len(livre) - 1):
        titre_livre = titre_livre + livre[i] + "+"
    titre_livre = titre_livre + livre[len(livre)-1]
    url = search_livraddict(titre_livre)
    infos = obtain_type(url)
    type_de_livre = type_livre(infos)
    return type_de_livre

In [17]:
def apply_search(row):
    nouveau = ["Nouveauté", "Nouveauté jeunesse"]
    if row["type_de_document"] in nouveau:
        parties = row["titre"].split(".")
        parties_bis = row["titre"].split(":")

        #pour les séries avec nom de série, numéro tome, nom du tome
        if len(parties)==3:
            saga = parties[0].strip()
            tome = parties[1].strip()
            nom = parties[2].strip()
            if int(tome) < 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
                if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                    type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
            elif int(tome)>= 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
        
        #pour les séries avec que nom de série et numéro de tome
        elif len(parties)==2:
            saga = parties[0].strip()
            tome = parties[1].strip()
            if int(tome) < 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
                if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                    type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
            elif int(tome)>= 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
        
        #pour les livre où il y a écrit le genre après le titre
        elif len(parties_bis) == 2:
            type_de_livre = search_type_livre(row["titre"])
            if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                nom = parties_bis[0].strip()
                type_de_livre = search_type_livre(nom)

        else:
            type_de_livre = search_type_livre(row["titre"])
        return type_de_livre


    else:
        return row["type_de_document"]

In [18]:
df_filtered['type_de_document'] = df_filtered.apply(apply_search, axis=1)

/tmp/ipykernel_13432/1561553421.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['type_de_document'] = df_filtered.apply(apply_search, axis=1)


In [19]:
df_filtered.head(6)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
5,73,Livre adulte,241.0,Chien 51 : roman,"Gaudé, Laurent"
6,87,Livre adulte,225.0,Le gosse : roman,"Olmi, Véronique"
7,109,Bande dessinée jeunesse,196.0,Attaque au clair de lune,"Oda, Eiichiro"


In [20]:
erreur_recherche = ["Nom du livre pas trouvé sur Livraddict"]
df_recherche =  df_filtered[df_filtered["type_de_document"].isin(erreur_recherche)]
print(df_recherche)

Empty DataFrame
Columns: [rang, type_de_document, reservations, titre, auteur]
Index: []


In [21]:
erreur_recherche = ["Introuvable"]
df_recherche =  df_filtered[df_filtered["type_de_document"].isin(erreur_recherche)]
print(df_recherche)

Empty DataFrame
Columns: [rang, type_de_document, reservations, titre, auteur]
Index: []


S'assurer que les livres soient bien dans l'ordre croissant de leur rang avant d'ajouter un numéro absolu

In [22]:
df_bibli = df_filtered.sort_values(by="rang")
df_bibli = df_bibli.reset_index(drop=True)

Ajouter une nouvelle variable permettant de supprimer les trous

In [23]:
df_bibli['Classement'] = df_bibli.index + 1

In [24]:
df_bibli.head(5)

,rang,type_de_document,reservations,titre,auteur,Classement
0,5,Livre adulte,755.0,La décision : roman,"Tuil, Karine",1
1,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona",2
2,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad",3
3,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad",4
4,25,Livre adulte,398.0,Dans les brumes de Capelans,"Norek, Olivier",5


TEST AVEC DES EXEMPLES SIMPLES

In [25]:
search_livraddict("Percy+Jackson")

'https://www.livraddict.com/biblio/livre/percy-jackson-comics-tome-1-le-voleur-de-foudre.html'

In [26]:
obtain_type("https://www.livraddict.com/biblio/livre/percy-jackson-comics-tome-1-le-voleur-de-foudre.html")

[' Comics', ' Adolescence']

In [27]:
type_livre([' Comics', ' Adolescence'])

'Bande dessinée ado'

In [28]:
parties = "Journal inquiet d'Istanbul. 1".split(".")
if len(parties)==2:
    saga = parties[0].strip()
    tome = parties[1].strip()
    if int(tome) < 10:
        type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
        if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
            type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
    elif int(tome)>= 10:
        type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)

In [29]:
print(type_de_livre)

Bande dessinée ado


In [30]:
"L'arbre gelé".split()

["L'arbre", 'gelé']

In [31]:
livre = "Il était par une nuit un voyageur".split()
titre_livre = ""
for i in range(0, len(livre) - 1):
    titre_livre = titre_livre + livre[i] + "+"
titre_livre = titre_livre + livre[len(livre)-1]

In [32]:
print(titre_livre)

Il+était+par+une+nuit+un+voyageur


In [33]:
titre = "Journal inquiet d'Istanbul. 1"
parties = titre.split(".")
parties_bis = titre.split(":")

#pour les séries avec nom de série, numéro tome, nom du tome
if len(parties)==3:
    saga = parties[0].strip()
    tome = parties[1].strip()
    nom = parties[2].strip()
    if int(tome) < 10:
        type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
        if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
            type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
    elif int(tome)>= 10:
        type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
        
#pour les séries avec que nom de série et numéro de tome
elif len(parties)==2:
    saga = parties[0].strip()
    tome = parties[1].strip()
    if int(tome) < 10:
        type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
        if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
            type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
    elif int(tome)>= 10:
        type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
        
#pour les livre où il y a écrit le genre après le titre
elif len(parties_bis) == 2:
    type_de_livre = search_type_livre(titre)
    if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
        nom = parties_bis[0].strip()
        type_de_livre = search_type_livre(nom)

else:
    type_de_livre = search_type_livre(row["titre"])
print(type_de_livre)

Bande dessinée ado


In [35]:
titre = "Chien 51: le roman"
parties = titre.split(".")
parties_bis = titre.split(":")
if len(parties_bis) == 2:
    type_de_livre = search_type_livre(titre)
    if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
        nom = parties_bis[0].strip()
        type_de_livre = search_type_livre(nom)
print(type_de_livre)

Livre adulte


In [ ]:
# On transforme en df le csv pour observer en entier

df_bibli.to_csv('bibli.csv', index=False, encoding='utf-8')

In [42]:
# on nettoie la table pour armoniser avec les autres et préparer la fusion 

#variable auteur 

# on renomme en auteur en Auteur 
df_bibli.rename(columns={"auteur": "Auteur"}, inplace=True)

# Au lieu d'écrire "Nom, Prénom" on veut sous format "Prénom Nom" : pour cela on crée une fonction
def inverser_nom_prenom(nom_complet):
    if pd.isna(nom_complet):
        return nom_complet
    parties = [part.strip() for part in nom_complet.split(",")]
    if len(parties) == 2:
        return f"{parties[1]} {parties[0]}"
    return nom_complet

# On applique la foncyion a toute la colonne 'Auteur'
df_bibli["Auteur"] = df_bibli["Auteur"].apply(inverser_nom_prenom)

# On vérifie que cela marche
print(df_bibli.head(10))


   rang       type_de_document  reservations  \
0     5           Livre adulte         755.0   
1    12           Livre adulte         522.0   
2    13  Bande dessinée adulte         521.0   
3    14  Bande dessinée adulte         510.0   
4    25           Livre adulte         398.0   
5    28           Livre adulte         367.0   
6    33           Livre adulte         345.0   
7    72           Livre adulte         246.0   
8    73           Livre adulte         241.0   
9    75           Livre adulte         239.0   

                                               titre                 Auteur  \
0                                La décision : roman            Karine Tuil   
1  Réinventer l'amour : comment le patriarcat sab...           Mona Chollet   
2                                    Le jeune acteur           Riad Sattouf   
3            Une jeunesse au Moyen-Orient, 1992-1994           Riad Sattouf   
4                        Dans les brumes de Capelans          Olivier Norek 

In [47]:
# On renomme la variable titre
df_bibli.rename(columns={"titre": "Titre"}, inplace=True)

print(df_bibli.columns)

# On met tout les titres en minuscule 
df_bibli['Titre'] = df_bibli['Titre'].str.strip().str.lower()

print(df_bibli.head(10))


Index(['rang', 'type_de_document', 'reservations', 'Titre', 'Auteur',
       'Classement'],
      dtype='object')
   rang       type_de_document  reservations  \
0     5           Livre adulte         755.0   
1    12           Livre adulte         522.0   
2    13  Bande dessinée adulte         521.0   
3    14  Bande dessinée adulte         510.0   
4    25           Livre adulte         398.0   
5    28           Livre adulte         367.0   
6    33           Livre adulte         345.0   
7    72           Livre adulte         246.0   
8    73           Livre adulte         241.0   
9    75           Livre adulte         239.0   

                                               Titre                 Auteur  \
0                                la décision : roman            Karine Tuil   
1  réinventer l'amour : comment le patriarcat sab...           Mona Chollet   
2                                    le jeune acteur           Riad Sattouf   
3            une jeunesse au moyen-orient

In [49]:
# On renomme la variable type de document catégorie
df_bibli.rename(columns={"type_de_document": "Catégorie"}, inplace=True)
 
print(df_bibli.columns)

Index(['rang', 'Catégorie', 'reservations', 'Titre', 'Auteur', 'Classement'], dtype='object')


In [53]:
# on crée une variable indicatrice pour dire que le livre est dans le classement
df_bibli['top_bibli'] = 1

print(df_bibli.head(10))


   rang              Catégorie  reservations  \
0     5           Livre adulte         755.0   
1    12           Livre adulte         522.0   
2    13  Bande dessinée adulte         521.0   
3    14  Bande dessinée adulte         510.0   
4    25           Livre adulte         398.0   
5    28           Livre adulte         367.0   
6    33           Livre adulte         345.0   
7    72           Livre adulte         246.0   
8    73           Livre adulte         241.0   
9    75           Livre adulte         239.0   

                                               Titre                 Auteur  \
0                                la décision : roman            Karine Tuil   
1  réinventer l'amour : comment le patriarcat sab...           Mona Chollet   
2                                    le jeune acteur           Riad Sattouf   
3            une jeunesse au moyen-orient, 1992-1994           Riad Sattouf   
4                        dans les brumes de capelans          Olivier Norek 

In [ ]:
#on fusionne avec la table livraddict/fnac/babelio/wikipedia si jamais on a deux catégories différentes on priorise celle de la premiere table 
